In [2]:
import pandas as pd

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import (
    AutoTokenizer,
    AutoConfig,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
    RobertaForMultipleChoice,
    AutoModelForSequenceClassification,
    LlamaModel,
    LlamaForSequenceClassification,
    BitsAndBytesConfig,
    get_polynomial_decay_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    TrainerCallback,
)
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset
from random import random, randint
from utils import load_json, load_split_data

In [ ]:
class args:
    train_data = './dataset/demo_train.csv'
    MAX_INPUT = 1024

In [ ]:
df_train = pd.read_csv(args.train_data).reset_index(drop = True)
#df_valid = pd.read_csv(args.valid_data).reset_index(drop = True)

In [ ]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

In [ ]:
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [ ]:
def preprocess(example):
    first_sentence = [ "[CLS] " + example['prompt'] ] * 2
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in ['response_a','response_b']]
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='longest_first', 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [ ]:
def preprocess(example):
    sentences = [" #### " + example['prompt'] + " [SEP] " + example['response_a'] + " [SEP]" +  " #### " + example['prompt'] + " [SEP] " + example['response_b'] + " [SEP]"]
    tokenized_example = tokenizer(sentences, truncation=True, 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [ ]:
dataset = datasets.Dataset.from_pandas(df_train)
MODEL = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
dataset

In [ ]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b'])# 

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset['input_ids'][0]

In [ ]:
df_train.loc[40]

In [ ]:
df_train.loc[:1000,].reset_index(drop = True).to_csv('demo_train.csv')

In [ ]:
df_train.loc[1000:1200,].reset_index(drop = True).to_csv('demo_valid.csv')

In [ ]:
import os

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice, AutoModelForSequenceClassification, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset
from sklearn.metrics import log_loss
import torch.nn as nn
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
    

In [ ]:
MODEL = 'meta-llama/llama-3-transformers-8b-hf-v1'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens({"pad_token":"<pad>"})

In [ ]:
tokenizer("<pad>")['input_ids'][0]

In [ ]:
128256 in tokenizer("<pad>")['input_ids']

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False
    )
    
#config = AutoConfig.from_pretrained(args.MODEL)
model = LlamaForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #config = config,
    device_map="auto")

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False
    )
    
#config = AutoConfig.from_pretrained(args.MODEL)
model = LlamaForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #config = config,
    device_map="auto")
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

# config = AutoConfig.from_pretrained(MODEL)
# config.hidden_dropout_prob = args.dropout_rate
# config.attention_probs_dropout_prob = args.dropout_rate
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,  # For sequence classification
#     inference_mode=False,
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     bias = 'none',
#     target_modules=["q_proj","k_proj","v_proj"]  # Target specific modules
# )
# model = get_peft_model(model, peft_config)

In [ ]:
for key in model.state_dict():
        print(f"{key}, {model.state_dict()[key].shape}, {model.state_dict()[key].dtype}")

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
[i.dtype for i in model.parameters()]

In [ ]:
from utils import load_split_data

In [ ]:
df_train, df_valid = load_split_data('dataset/1k_mt_bench_human_judgments.json', 1, 3000, True, False)

In [ ]:
df_train, df_valid = load_split_data('dataset/lmsys-chatbot_arena_conversations-33k.csv', 2, 3000, True, False)

In [ ]:
df_valid

In [ ]:
df_train

In [ ]:
idx = 1
prompt_response = df_train.loc[idx,'prompt_response']
label = df_train.loc[idx,'label']

In [ ]:
print(prompt_response)
print("")
print(label)

In [ ]:
prompt_response

In [ ]:
tokenizer.decode([1,
 32006,
 887,
 526])

In [ ]:
tokenizer.decode([887])

In [ ]:
tokenizer.encode('<|system|>\nYou')

In [ ]:
tokenizer.encode('Apple\nBa')

In [ ]:
tokenizer.decode([396,
 18571,
 415,
 13,
 4548,
 7420,])

In [ ]:
tokenizer.decode([29933])

In [ ]:
print(templete_part1 + prompt_response + templete_part2 + templete_part3 + label)

In [ ]:
templete_part1 = "<|system|>\nYou are a helpful assistant good at judging conversations.<|end|>\n<|user|>\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n"
templete_part1_input_ids = tokenizer(text=templete_part1, add_special_tokens=True, padding=False)['input_ids']

templete_part2 = "\n###options\nA. Model A\nB. Model B\nC. Tie\n<|end|>\n"
templete_part2_input_ids = tokenizer(text=templete_part2, add_special_tokens=True, padding=False)['input_ids'][1:]

templete_part3 = "<|assistant|>\n"
templete_part3_input_ids = tokenizer(text=templete_part3, add_special_tokens=True, padding=False)['input_ids'][1:]

prompt_response_ids = tokenizer(text=prompt_response, add_special_tokens=True, truncation=True,
                                      max_length=3000, padding=False)['input_ids'][1:]


label_ids = tokenizer.encode(text=label, add_special_tokens=False)
input_ids = templete_part1_input_ids + prompt_response_ids + templete_part2_input_ids + templete_part3_input_ids + label_ids + [tokenizer.eos_token_id]
print(tokenizer.decode(input_ids))

In [ ]:
text = "Apple"
prompt_response = templete_part1 + text + templete_part2 + templete_part3 + label + tokenizer.eos_token
print(prompt_response)

In [ ]:
MODEL = 'microsoft/LLM-Research/Phi-3-mini-4k-instruct'
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True, truncation_side = 'left')

In [ ]:
tokenizer(prompt_response)

In [ ]:
tokenizer.eos_token_id

In [ ]:
tokenizer.eos_token

In [ ]:
tokenizer.bos_token

In [ ]:
tokenizer.decode([887])

In [ ]:
tokenizer('A',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
tokenizer.pad_token_id

In [ ]:
tokenizer.pad_token

In [ ]:
tokenizer('<|user|>',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
AutoModelForCausalLM.from_pretrained(MODEL)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
print(text)

In [ ]:
templete_part1 = "<|im_start|>system\nYou are a helpful assistant good at judging conversations.<|im_end|>\n<|im_start|>user\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n"
templete_part1_input_ids = tokenizer(text=templete_part1, add_special_tokens=True, padding=False)['input_ids']

templete_part2 = "\n###options\nA. Model A\nB. Model B\nC. Tie\n<|im_end|>\n"
templete_part2_input_ids = tokenizer(text=templete_part2, add_special_tokens=True, padding=False)['input_ids']
#print(f"templete_part2 is {templete_part2_input_ids}")
templete_part3 = "<|im_start|>assistant\n"
templete_part3_input_ids = tokenizer(text=templete_part3, add_special_tokens=True, padding=False)['input_ids']

prompt_response_ids = tokenizer(text=prompt_response, add_special_tokens=True, truncation=True,
                                      max_length=3000, padding=False)['input_ids']


label_ids = tokenizer.encode(text=label, add_special_tokens=False)
input_ids = templete_part1_input_ids + prompt_response_ids + templete_part2_input_ids + templete_part3_input_ids + label_ids + [tokenizer.eos_token_id]
print(tokenizer.decode(input_ids))

In [ ]:
tokenizer.decode(14374)

In [ ]:
tokenizer.pad_token,tokenizer.eos_token,

In [ ]:
tokenizer('A',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
MODEL = 'Qwen/Qwen2-7B-Instruct'
config = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True, truncation_side = 'left')
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )
model = AutoModelForCausalLM.from_pretrained(MODEL,
                                             config=config,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             attn_implementation='eager')

In [ ]:
model

In [ ]:
tmp = pd.read_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample14k.json")
#tmp2 = pd.read_json("dataset/lmsys-chatbot_arena_conversations-33k.json")

In [ ]:
tmp = tmp.drop(columns = ['difference'])

In [ ]:
t = pd.concat([tmp,tmp2]).reset_index(drop = True)

In [ ]:
t.prompt[0]

In [ ]:
is_english(t.prompt[0][0])

In [ ]:
is_english(t.prompt[46969][0])

In [ ]:
from langdetect import detect

# 检测语言并过滤非英文行
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

tmp = tmp[tmp['prompt'].apply(lambda x: is_english(x[0]))]

In [ ]:
tmp = tmp.reset_index(drop = True)

In [ ]:
tmp.to_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample14k.json")

In [ ]:
ex = pd.read_csv("dataset/kaggle-ultrafeedback-drop-duplicate.csv")
tie = pd.read_csv("dataset/kaggle-ultrafeedback-ties-drop-duplicate.csv")
p = pd.read_csv("dataset/ultrafeedback_prediction.csv")

from utils import load_json
ex = load_json(ex)
tie = load_json(tie)

In [ ]:
total = pd.concat([tie,ex]).reset_index(drop = True)

In [ ]:
p

In [ ]:
p = p.rename(columns = {'winner_model_a':"p_winner_model_a", 'winner_model_b':"p_winner_model_b",  'winner_tie':"p_winner_tie"})

In [ ]:
final = pd.concat([total, p], axis = 1)

In [ ]:
final

In [ ]:
def get_p_label(row):
    a = row.p_winner_model_a
    b = row.p_winner_model_b
    c = row.p_winner_tie

    l = [a ,b, c]
    label = l.index(max(l))
    return label

In [ ]:
final['p_label'] = final.apply(get_p_label, axis = 1)

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

final['label'] = final.apply(get_label, axis = 1)

In [ ]:
filter = final.loc[final.p_label == final.label,:].reset_index(drop = True)

In [ ]:
threshold = 0.9
filter_list = (filter.p_winner_model_a >= threshold) | (filter.p_winner_model_b >= threshold) | (filter.p_winner_tie >= threshold)
filter = filter.loc[filter_list,:].reset_index(drop = True)

In [ ]:
filter.prompt.values[0][0]

In [ ]:
from langdetect import detect

# 检测语言并过滤非英文行
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

filter_only_english = filter[filter['prompt'].apply(lambda x: is_english(x[0][:30]))].reset_index(drop = True)

In [ ]:
save_columns = ['prompt', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'response_a', 'response_b', 'id']
filter[save_columns].to_json(f"dataset/70k_filter_threshold{threshold}.json", index = False)
filter_only_english[save_columns].to_json(f"dataset/70k_filter_only_english_threshold{threshold}.json", index = False)

In [ ]:
filter[save_columns]

In [ ]:
filter.label.value_counts()

In [ ]:
df_train, df_valid = load_split_data('dataset/train.csv', 2, 3000, True, False)

In [ ]:
df_train

In [ ]:
print(df_train.prompt_response.values[0])

In [ ]:
print(df_train.prompt_response.values[1])

In [ ]:
print(df_train.label.values[1])

In [ ]:
check = pd.read_json("dataset/70k_filter_only_english_threshold0.9.json")

In [ ]:
check.prompt.values[0]

In [ ]:
data_path = "dataset/kaggle-ultrafeedback-drop-duplicate-sample20k_least_similar_by_prompt_same_prediction_thr90.csv"
t = pd.read_csv(data_path)
t['id'] = [randint(10000,99999) + i for i in range(len(t))]
t.to_csv(data_path, index=False)

In [ ]:
t = pd.read_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample20k_most_similar_by_prompt_same_prediction_thr90.json")

In [48]:
#
train , valid = load_split_data('dataset/train.csv', 2, 2300, True, True, False)

100%|██████████| 3552/3552 [00:00<00:00, 13385.84it/s]


In [4]:
train , valid = load_split_data('dataset/train_sample10k_switch.json', 2, 2300, True, False, False)

100%|██████████| 12423/12423 [00:01<00:00, 9760.78it/s]


In [ ]:
train_id = train.id.to_list()
valid_id = valid.id.to_list()

In [ ]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

In [ ]:
data['prompt_str'] = data['prompt'].astype(str)

In [ ]:
from sklearn.model_selection import train_test_split
unique_prompts = data['prompt_str'].unique()
train_prompts, valid_prompts = train_test_split(unique_prompts, test_size=0.1, random_state=42)

In [ ]:
train_prompts_set = set(train_prompts)
valid_prompts_set = set(valid_prompts)

In [ ]:
# 根据划分的 prompt 获取对应的行
train = data[data['prompt_str'].isin(train_prompts_set)]
valid = data[data['prompt_str'].isin(valid_prompts_set)]

In [ ]:
train = data.loc[data.id.isin(train_id)].reset_index(drop = True)
valid = data.loc[data.id.isin(valid_id)].reset_index(drop = True)

In [ ]:
[i for i in train_id if i in valid_id]

In [ ]:
len(train_id)

In [ ]:
t = pd.read_json('dataset/train_sample10k_switch.json')

In [ ]:
len([i for i in t.prompt.values.tolist() if i in valid.prompt.values.tolist()])

In [ ]:
search = [['I read 60 pages of a book on Monday and 1/4 of the book on Tuesday. I completed the remaining 1/8 of the book on Wednesday. How many total pages are in the book?']]
train.loc[train.prompt.isin(search)]

In [ ]:
valid.loc[valid.prompt.isin(search)]

In [ ]:
[i for i in t.prompt.values.tolist() if i in valid.prompt.values.tolist()]

In [ ]:
len([i for i in train.prompt.values.tolist() if i in valid.prompt.values.tolist()])

In [ ]:
t.prompt.values.tolist()[0]

In [118]:
train , valid = load_split_data('dataset/train.csv', 2, 2300, True, True, False)
valid_id = valid.id.tolist()
train_id = train.id.tolist()

100%|██████████| 3552/3552 [00:00<00:00, 13074.61it/s]


In [119]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

In [52]:
s = data.loc[data.id.isin(train_id)]

In [30]:
s = s.sample(10000).reset_index(drop = True)

In [31]:
s_id = s.id.to_list()
len([i for i in s_id if i in valid_id])

0

In [ ]:
tmp_valid = data.loc[data.id.isin(valid_id)].reset_index(drop = True)
[i for i in s.prompt.values.tolist() if i in tmp_valid.prompt.values.tolist()]

In [122]:
ex_33 = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')
ex_33 = load_json(ex_33)

In [148]:
idx = [idx for idx, i in enumerate(tmp_valid.prompt.values.tolist()) if i in ex_33.prompt.values.tolist()]

In [164]:
same_prompt_in_valid = tmp_valid.iloc[idx,:].reset_index(drop = True)
not_same_prompt_in_valid = tmp_valid.iloc[~tmp_valid.index.isin(idx),:].reset_index(drop = True)

In [165]:
assert len(not_same_prompt_in_valid) + len(same_prompt_in_valid) == len(tmp_valid)

In [167]:
same_prompt_in_valid.to_json("dataset/same_prompt_in_valid.json", index = False)
not_same_prompt_in_valid.to_json("dataset/not_same_prompt_in_valid.json", index = False)

In [178]:
same_prompt_in_valid.sort_values(by = ['prompt'])

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
346,3434854914,koala-13b,gpt-4-0314,[\n\n.. / .-.. .. -.- . / -.-- --- ..- / ---.....,[This is a character map for the IBM Selectric...,"[Your Morse code message translates to: ""I LIK...",0,1,0
250,2463216319,claude-instant-1,vicuna-7b,[\n### Instructions:\nDecide what should answe...,[Bob should answer:\n\nHour is out of correct ...,[Bob means that the date and time range given ...,0,0,1
273,2622512543,vicuna-13b,vicuna-7b,[\npoint out the typora.\n\n \caption{ N...,[The typora is the text that is being generate...,[The typora refers to the table in the text th...,0,0,1
394,3931418201,vicuna-33b,yi-34b-chat,"[ Five people were eating apples, A finished b...","[Based on the information provided, we can det...",[Let's break down the information given:\n\n1....,0,0,1
243,2382196251,alpaca-13b,mpt-7b-chat,"[""But we can also hack these reward centers fo...",[Humans and other animals can hack their rewar...,[Play is an essential aspect of the human and ...,0,1,0
...,...,...,...,...,...,...,...,...,...
143,1348660373,fastchat-t5-3b,RWKV-4-Raven-14B,[write the lyrics of a taylor swift song about...,"[I'm sorry, but I cannot write lyrics to a son...",[Verse 1:\nI was running late for work one day...,0,1,0
238,2318149804,vicuna-13b,llama-13b,[you are a world class investment analyst work...,"[As a language model, I don't have personal op...",[Here are some sectors that are ripe for a rol...,1,0,0
98,900774276,dolly-v2-12b,koala-13b,"[you are tasked with taking a question, or lis...",[You can pay cash into your account by using a...,"[{""sample"": ""Can you make a bank transaction i...",0,1,0
264,2570827944,vicuna-13b,oasst-pythia-12b,[“This core competency enables Foo to perform ...,[1. Diverse range of abilities\n2. Multifacete...,[An array of functions can be phrased in sever...,1,0,0


In [180]:
idx_ex = [idx for idx, i in enumerate(ex_33.prompt.values.tolist()) if i in tmp_valid.prompt.values.tolist()]

In [181]:
ex_33.iloc[idx_ex,:].reset_index(drop = True).sort_values(by = ['prompt'])

,id,model_a,model_b,winner_model_a,winner_model_b,winner_tie,prompt,response_a,response_b
323,adc21d914ed04996b86144db1a1fcdb9,gpt-4,koala-13b,1,0,0,[\n\n.. / .-.. .. -.- . / -.-- --- ..- / ---.....,"[Your Morse code message translates to: ""I LIK...",[This is a character map for the IBM Selectric...
608,dedc4c7fbb574ae0bec904b3d2207250,mpt-7b-chat,palm-2,0,1,0,[\n### Instructions:\nDecide what should answe...,[Bob: The date and time are in correct range. ...,"[Bob should say: ""18:81 is not a valid time.""]"
610,5712ebbf2ed24b128fce9ff8b0c75f76,fastchat-t5-3b,koala-13b,0,0,1,[\n### Instructions:\nDecide what should answe...,[Alice: The date and time are incorrect. The c...,[Bob: 8 December 18:81]
609,0803b97a4f3f45989e6320208c92c6da,vicuna-7b,claude-instant-v1,0,0,1,[\n### Instructions:\nDecide what should answe...,[Bob means that the date and time range given ...,[Bob should answer:\n\nHour is out of correct ...
635,21fb7343633c482d9e7cacbe8c5c8568,oasst-pythia-12b,fastchat-t5-3b,0,0,1,[\npoint out the typora.\n\n \caption{ N...,[The Typora is a text editor developed by Face...,[There are two typos in this passage:\n* In th...
...,...,...,...,...,...,...,...,...,...
343,0894e88f8b73486eac6383b943c28fe6,koala-13b,fastchat-t5-3b,1,0,0,"[•\tTo solve math problems, you should use the...","[1. To solve the problem, we need to use the...","[To calculate the expression 10/5+2*2*2=14, we..."
347,149a197246934f3ab1c49fe0f2a248e3,koala-13b,gpt-3.5-turbo,0,1,0,"[•\tTo solve math problems, you should use the...",[10/5 + 2 \* 2 \* 2 = (10 / 5) + (2 \* 2 \* 2)...,"[Using PEMDAS, we need to do multiplication be..."
346,6d5dc43f983e4eaca6f03b16d65d9bae,claude-v1,RWKV-4-Raven-14B,1,0,0,"[•\tTo solve math problems, you should use the...","[Okay, let's solve this step-by-step using the...","[To calculate 10/5 + 2*2*2, typesubmit it to y..."
341,4d2f8be7bd6b4d519ea921a3009e0680,claude-v1,koala-13b,1,0,0,"[•\tTo solve math problems, you should use the...","[Okay, let's use the order of operations (PEMD...","[To solve the problem, you should use the orde..."


In [38]:
s['id'] = [randint(100000,999999) + i for i in range(len(s))]

In [39]:
def switch(row):
    response_a = row.response_a
    response_b = row.response_b

    row.response_a = response_b
    row.response_b = response_a
    
    return row

In [40]:
s = s.apply(switch, axis = 1)
label_switch = {0:1, 1:0}
s.loc[s.winner_tie !=1, 'winner_model_a'] = s.loc[s.winner_tie !=1, 'winner_model_a'].map(label_switch)
s.loc[s.winner_tie !=1, 'winner_model_b'] = s.loc[s.winner_tie !=1, 'winner_model_b'].map(label_switch)

In [41]:
s.to_json("dataset/train_sample10k_switch.json", index = False)

In [42]:
check = pd.read_json("dataset/train_sample10k_switch.json")

In [43]:
check

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,726706,gpt-3.5-turbo-0613,yi-34b-chat,[<|Instruction|>: Use the <|QUESTION|> and <|C...,[<|ANSWER|>: Patient reports he/she uses alcoh...,[Patient reports he/she uses alcohol or other ...,0,0,1
1,140571,claude-2.1,gpt-4-0613,[How do residential proxy botnets work?],[A residential proxy botnet is a network of in...,"[I apologize, I should not provide information...",1,0,0
2,367729,vicuna-33b,yi-34b-chat,[Write a brief analysis on what economic polic...,"[Eritrea, a country in the Horn of Africa, has...","[Eritrea, a small country in the Horn of Afric...",0,0,1
3,741232,RWKV-4-Raven-14B,koala-13b,[Do you have any tips for a new freelancer sta...,"[Yes, here are a few tips for a new freelancer...","[Yes, here are some tips for new freelancers o...",1,0,0
4,985688,llama-2-13b-chat,gpt-4-0613,[Yoooo],[Hello! How can I assist you today?],[Hello there! I'm here to help answer any ques...,1,0,0
...,...,...,...,...,...,...,...,...,...
9995,656488,claude-1,openchat-3.5,[famous fictional internet name consisting of ...,[The answer is EPEP.],[Here is a famous 4-letter fictional internet ...,0,0,1
9996,545964,mistral-medium,gpt-4-1106-preview,[What is Genshin Impact? Introduce it to me in...,[Genshin Impact is a free-to-play action role-...,[Genshin Impact is a popular open-world action...,1,0,0
9997,294477,claude-2.0,alpaca-13b,[Hi! Can I ask you anything?],"[Sure, what can I help you with?]","[Sure, feel free to ask me anything!]",0,0,1
9998,667805,mixtral-8x7b-instruct-v0.1,claude-1,[Does the Genesee river flow north or south?],[The Genesee River flows northward. It origina...,"[The Genesee River, which flows through New Yo...",1,0,0


In [47]:
print(check.prompt.values[1][0])

How do residential proxy botnets work?


In [ ]:
t.loc[t.response_a == t.response_b, 'winner_tie'].value_counts()

In [ ]:
t.loc[t.response_a=='["Hyderabad"]', ]

In [ ]:
t.loc[(t.response_a == '[null]')]

In [ ]:
t.loc[(t.response_a == t.response_b) & (t.winner_tie != 1),]

In [ ]:
t.loc[(t.response_a == t.response_b)]

In [ ]:
t.loc[3844:3847,:]

In [3]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

ex_33 = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')
ex_33 = load_json(ex_33)

In [172]:
'''
1、找出train里面不与33k重复部分
2、不重复的部分再划分
'''



In [4]:
set_prompt_response = []
for i in data.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
data['set_prompt_response'] = set_prompt_response    

In [5]:
set_prompt_response = []
for i in ex_33.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
ex_33['set_prompt_response'] = set_prompt_response  

In [6]:
idx = [idx for idx, i in enumerate(data.set_prompt_response.values) if i in ex_33.set_prompt_response.values]

In [7]:
same = data.loc[idx,:].reset_index(drop = True)

In [8]:
ex_33.loc[ex_33.set_prompt_response == same.set_prompt_response.values[0],:]

,id,model_a,model_b,winner_model_a,winner_model_b,winner_tie,prompt,response_a,response_b,set_prompt_response
12419,edb4eac598f848e19c8996c5ca150fda,koala-13b,gpt-3.5-turbo,0,1,0,[What is the best way to travel from Tel-Aviv ...,[The best way to travel from Tel Aviv to Jerus...,[The best way to travel from Tel-Aviv to Jerus...,{The best way to travel from Tel-Aviv to Jerus...


In [9]:
not_same = data.loc[~data.index.isin(idx),:].reset_index(drop = True)

In [10]:
assert len(not_same) + len(same) == len(data)

In [11]:
len([idx for idx, i in enumerate(not_same.set_prompt_response.values) if i in ex_33.set_prompt_response.values])

0

In [33]:
len([idx for idx, i in enumerate(not_same.set_prompt_response.values) if i in same.set_prompt_response.values])

0

In [26]:
unique_sets = not_same['set_prompt_response'].drop_duplicates().reset_index(drop=True)
# 将唯一集合进行随机划分
unique_sets = unique_sets.sample(frac=1, random_state=42).reset_index(drop=True)
midpoint = len(unique_sets) // 10
set1 = unique_sets.iloc[:midpoint]
set2 = unique_sets.iloc[midpoint:]

In [51]:
# 根据划分结果从原数据集中提取对应的行
valid = not_same[not_same['set_prompt_response'].isin(set1)].reset_index(drop=True)
train_subset = not_same[not_same['set_prompt_response'].isin(set2)].reset_index(drop=True)
assert len(valid) + len(train_subset) == len(not_same)
assert len(valid) + len(train_subset) + len(same) == len(data)

In [52]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_subset.set_prompt_response.values])

0

In [53]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in same.set_prompt_response.values])

0

In [54]:
train_exclude_valid = pd.concat([train_subset, same]).reset_index(drop=True) #train 里面排除valid
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values])

0

In [55]:
assert len(valid) + len(train_exclude_valid) == len(data)

In [60]:
train_33k = pd.concat([train_subset, ex_33]).reset_index(drop=True)

In [38]:
train_subset = train.drop(columns = ['set_prompt_response'])
valid = valid.drop(columns = ['set_prompt_response'])
train_exclude_valid = train_exclude_valid.drop(columns = ['set_prompt_response'])
train_33k = train_33k.drop(columns = ['set_prompt_response'])

In [56]:
train_subset.to_json("dataset/non_overlap/train_subset.json", index = False)
valid.to_json("dataset/non_overlap/valid.json", index = False)
train_exclude_valid.to_json("dataset/non_overlap/train_exclude_valid.json", index = False)
train_33k.to_json("dataset/non_overlap/train_33k.json", index = False)

In [64]:
# 检查
train_subset = pd.read_json("dataset/non_overlap/train_subset.json")
valid = pd.read_json("dataset/non_overlap/valid.json")
train_exclude_valid = pd.read_json("dataset/non_overlap/train_exclude_valid.json")
train_33k = pd.read_json("dataset/non_overlap/train_33k.json")

In [65]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [66]:
train_subset = get_set_prompt_response(train_subset)
valid = get_set_prompt_response(valid)
train_exclude_valid = get_set_prompt_response(train_exclude_valid)
train_33k = get_set_prompt_response(train_33k)

In [68]:
#valid和任何都不重合
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_subset.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_33k.set_prompt_response.values]) == 0

In [73]:
train_33k.prompt.values[0]

['Is it morally right to try to have a certain percentage of females on managerial positions?',
 'OK, does pineapple belong on a pizza? Relax and give me fun answer.']

In [217]:
# 提取唯一的 prompt 进行划分
not_same['prompt_str'] = not_same['prompt'].astype(str)
unique_prompts = data['prompt_str'].unique()
train_prompts, valid_prompts = train_test_split(unique_prompts, test_size=0.1, random_state=42)

train_prompts_set = set(train_prompts)
valid_prompts_set = set(valid_prompts)

# 根据划分的 prompt 获取对应的行
train = data[data['prompt_str'].isin(train_prompts_set)].reset_index(drop = True)
valid = data[data['prompt_str'].isin(valid_prompts_set)].reset_index(drop = True)
train = train.drop(columns = ['prompt_str'])
valid = valid.drop(columns = ['prompt_str'])

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,set_prompt_response
0,30192,gpt-4-1106-preview,gpt-4-0613,[Is it morally right to try to have a certain ...,[The question of whether it is morally right t...,"[As an AI, I don't have personal beliefs or op...",1,0,0,{The question of whether it is morally right t...
1,53567,koala-13b,gpt-4-0613,[What is the difference between marriage licen...,[A marriage license is a legal document that a...,[A marriage license and a marriage certificate...,0,1,0,{If you want to get both a marriage license an...
2,65089,gpt-3.5-turbo-0613,mistral-medium,[explain function calling. how would you call ...,[Function calling is the process of invoking o...,[Function calling is the process of invoking a...,0,0,1,{Function calling is the process of invoking a...
3,96401,llama-2-13b-chat,mistral-7b-instruct,[How can I create a test set for a very rare c...,[Creating a test set for a very rare category ...,[When building a classifier for a very rare ca...,1,0,0,{How can I create a test set for a very rare c...
4,313413,mixtral-8x7b-instruct-v0.1,vicuna-13b,[Why water is not used in bath tub?],[Water is actually used in a bath tub. A bath ...,[Water is not used in a bath tub because it is...,1,0,0,{Water is actually used in a bath tub. A bath ...
...,...,...,...,...,...,...,...,...,...,...
49391,4294633312,pplx-7b-online,gpt-4-0613,[abc],[It seems like you're referring to the Califor...,"[Hello! It seems like you've typed ""abc."" How ...",1,0,0,"{abc, It seems like you're referring to the Ca..."
49392,4294656694,gpt-4-0613,claude-1,"[A simple mnemonic for π:\n""How I wish I could...","[Sure, let's break it down:\n\n1. ""How"" has 3 ...",[Here is how that mnemonic represents the digi...,1,0,0,"{A simple mnemonic for π:\n""How I wish I could..."
49393,4294692063,claude-2.0,llama-2-13b-chat,"[In python, implement a naive Bayes with gauss...",[Here is an implementation of a naive Bayes cl...,[Sure! Here's an implementation of a naive Bay...,1,0,0,{Sure! Here's an implementation of a naive Bay...
49394,4294899228,palm-2,tulu-2-dpo-70b,"[If a bait contains 0,0025% bromadiolon then h...",[Bromadiolone is a rodenticide which is most o...,"[As an AI language model, I do not promote or ...",0,1,0,{Bromadiolone is a rodenticide which is most o...
